In [1]:
%%capture --no-stderr
!pip install langchain langchain-openai langchain-core langchain-community
!pip install langsmith

In [2]:
import os
from google.colab import userdata

In [3]:
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGSMITH_KEY')
os.environ["LANGCHAIN_PROJECT"] = "notebook-creation-langchain"

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-nano")

In [5]:
topics = "dicionarios, listas e tuplas"

In [6]:
prompt = """
#################### Contexto ####################

Você é um assistente para a geração de conteúdo, seu papel é descrever
no formato de subtópicos os tópicos apresentados a seguir para que uma lista
de exercícios prática seja criada para a fixação do conteúdo da disciplina.

Enumere os principais e mais importantes subtópicos para o conteúdo a seguir:
{topics}
"""

In [7]:
from langchain_core.prompts import ChatPromptTemplate

In [8]:
template = ChatPromptTemplate.from_template(prompt)

In [9]:
chain = template | llm

In [10]:
response = chain.invoke(
    {
        "topics": topics
    }
)

In [13]:
print(response.content)

Claro! Aqui estão os principais e mais importantes subtópicos para a disciplina de **Dicionários, Listas e Tuplas**:

### Dicionários
- Definição e conceito de dicionários
- Estrutura de um dicionário (chave-valor)
- Criando dicionários (literais e funções construtoras)
- Acessando elementos (por chaves)
- Adicionando, modificando e removendo itens
- Métodos de dicionários (keys(), values(), items(), get(), pop(), update(), clear())
- Iterando sobre dicionários
- Uso prático de dicionários (exemplos de aplicações)

### Listas
- Definição e conceito de listas
- Criando listas (literais e funções construtoras)
- Acessando elementos (por índice)
- Modificando elementos
- Métodos de listas (append(), insert(), remove(), pop(), sort(), reverse(), extend(), clear())
- Fatiamento de listas
- Iterando sobre listas
- Uso prático de listas (exemplos de aplicações)

### Tuplas
- Definição e conceito de tuplas
- Diferenças entre tuplas e listas
- Criando tuplas (literais e funções construtoras)
- 

In [29]:
from pydantic import BaseModel, Field, field_validator

In [12]:
from enum import Enum

In [13]:
class CellTypes(Enum):
    markdown = 'markdown'
    code = 'code'

In [14]:
class NotebookCell(BaseModel):
    #metadata: dict = Field(description='notebook cell metadata')
    cell_type: CellTypes = Field(description='notebook cell type')

In [83]:
class CodeCell(NotebookCell):
    source: list[str] = Field(
        description='List of Python code lines to exemply the theorical content'
        )

    @field_validator('source')
    @classmethod
    def validate_and_split_strings(cls, value: list[str]) -> list[str]:
        """
        Validates if the input is a list with a single string containing newlines.
        If so, splits the string by newlines and returns the resulting list.
        """
        if len(value) == 1 and '\n' in value[0]:
            return [line + '\n' for line in value[0].split('\n') if len(line) > 0]

        return value


class MarkdownCell(NotebookCell):
    source: list[str] = Field(
        description='List containing topic name and explanation about what it is.'
        )

    @field_validator('source')
    @classmethod
    def validate_and_split_strings(cls, value: list[str]) -> list[str]:
        """
        Validates if the input is a list with a single string containing newlines.
        If so, splits the string by newlines and returns the resulting list.
        """
        if len(value) == 1 and '\n' in value[0]:
            return [line + '\n' for line in value[0].split('\n') if len(line) > 0]

        return value

class Content(BaseModel):
    markdown: MarkdownCell = Field(description='markdown text explaining the content')
    code: CodeCell = Field(description='python code to exemplify the content')

In [84]:
prompt_notebook = """
#################### Contexto ####################

Você é um assistente para a criação de conteúdos didáticos para uma disciplina.
Seu papel é gerar o conteúdo teórico e o código em python para serem adicionados
em um notebook como forma de exemplificar o que o aluno precisa aprender.

Gere sempre um par de células, sendo uma célula em markdown com o conteúdo explicativo
e outra célula com o código python, exemplificando claramente o conteúdo.

Orientações:
- Para as células em markdown adicione sempre um título resumindo o tema abordado em poucas palavras
- Use # para definir o título e coloque em negrito
- A explicação do conteudo coloque como um texto normal, podendo usar listas e/ou bullet point
- Gere o texto de explicação sempre em {language}
- Para as células de código python, gere um código que exemplifique ou explique o tópico abordado
- Todos os códigos deverão ser gerados em inglês e com nomes significativos para as variáveis

Gere as células em markdown e código python para o seguinte tópico {topic} que pertence ao tema {content}
"""

In [85]:
prompt_notebook_template = ChatPromptTemplate.from_template(prompt_notebook)

In [86]:
structured_llm = llm.with_structured_output(Content)

In [87]:
notebook_chain = prompt_notebook_template | structured_llm

In [88]:
response = notebook_chain.invoke(
    {
        "language": "pt-BR",
        "content": "Dicionários",
        "topic": "Definição e criação de dicionários"
    }
)

In [92]:
response.markdown.source

['# **Definição e Criação de Dicionários**\n',
 'Dicionários são estruturas de dados em Python que armazenam pares de chave-valor. Eles são úteis para representar informações que requerem uma associação entre um identificador único (chave) e um dado (valor). Algumas características importantes dos dicionários:\n',
 '- Cada chave deve ser única e imutável (como strings, números ou tuplas imutáveis).\n',
 '- Os valores podem ser de qualquer tipo de dado.\n',
 '- Os dicionários são mutáveis, ou seja, podem ser alterados após sua criação.\n',
 'Para criar um dicionário, usamos chaves {} e associamos chaves e valores usando dois pontos :.\n',
 'Vamos ver como criar e acessar dicionários.\n']

In [90]:
response.code.source

['# Criando um dicionário de exemplo\n',
 'student_info = {\n',
 '    "nome": "João",\n',
 '    "idade":  twenty-five,\n',
 '    "cursos": ["Matemática", "Física"],\n',
 '    "matricula": 123456\n',
 '}\n',
 '# Acessando elementos do dicionário\n',
 'print("Nome:", student_info["nome"])\n',
 'print("Idade:", student_info["idade"])\n',
 'print("Cursos:", student_info["cursos"])\n',
 'print("Matrícula:", student_info["matricula"])\n',
 '# Como modificar um valor\n',
 'student_info["idade"] = 26\n',
 'print("Idade atualizada:", student_info["idade"])\n',
 '# Adicionando um novo par chave-valor\n',
 'student_info["nota_final"] = 9.5\n',
 'print("Informações completas:", student_info)\n']

In [93]:
generated_cell = {
    "cells": [
        {
            "cell_type": "markdown",
            "source":
                response.markdown.source
            ,
            "metadata": {
                "id": ""
            }
        },
        {
            "cell_type": "code",
            "execution_count": None,
            "metadata": {
                "id": ""
            },
            "outputs": [],
            "source": response.code.source
        }
        ]
}

In [94]:
generated_cell

{'cells': [{'cell_type': 'markdown',
   'source': ['# **Definição e Criação de Dicionários**\n',
    'Dicionários são estruturas de dados em Python que armazenam pares de chave-valor. Eles são úteis para representar informações que requerem uma associação entre um identificador único (chave) e um dado (valor). Algumas características importantes dos dicionários:\n',
    '- Cada chave deve ser única e imutável (como strings, números ou tuplas imutáveis).\n',
    '- Os valores podem ser de qualquer tipo de dado.\n',
    '- Os dicionários são mutáveis, ou seja, podem ser alterados após sua criação.\n',
    'Para criar um dicionário, usamos chaves {} e associamos chaves e valores usando dois pontos :.\n',
    'Vamos ver como criar e acessar dicionários.\n'],
   'metadata': {'id': ''}},
  {'cell_type': 'code',
   'execution_count': None,
   'metadata': {'id': ''},
   'outputs': [],
   'source': ['# Criando um dicionário de exemplo\n',
    'student_info = {\n',
    '    "nome": "João",\n',
 

In [95]:
import json

In [96]:
print(json.dumps(generated_cell["cells"], indent=4, ensure_ascii=False))

[
    {
        "cell_type": "markdown",
        "source": [
            "# **Definição e Criação de Dicionários**\n",
            "Dicionários são estruturas de dados em Python que armazenam pares de chave-valor. Eles são úteis para representar informações que requerem uma associação entre um identificador único (chave) e um dado (valor). Algumas características importantes dos dicionários:\n",
            "- Cada chave deve ser única e imutável (como strings, números ou tuplas imutáveis).\n",
            "- Os valores podem ser de qualquer tipo de dado.\n",
            "- Os dicionários são mutáveis, ou seja, podem ser alterados após sua criação.\n",
            "Para criar um dicionário, usamos chaves {} e associamos chaves e valores usando dois pontos :.\n",
            "Vamos ver como criar e acessar dicionários.\n"
        ],
        "metadata": {
            "id": ""
        }
    },
    {
        "cell_type": "code",
        "execution_count": null,
        "metadata": {
      